### !!!This script take roughly an hour to run!!!

In [31]:
import pickle
from gensim.models.doc2vec import Doc2Vec,TaggedDocument

## Load in word model

In [9]:
# f = open("word_model_Doc2Vec.pickle","rb")
# word_model = pickle.load(f)
# f.close()

unable to import 'smart_open.gcs', disabling that module


## Load corpus

In [16]:
f = open("clean_text_with_punc.pickle","rb")
df = pickle.load(f)
f.close()
df.head()

,sentence,author
0,"[eugenie, grandet, by, honore, de, balzac, tra...","Balzac, Honoré de"
1,"[may, your, name,, that, of, one, whose, portr...","Balzac, Honoré de"
2,"[de, balzac.]","Balzac, Honoré de"
3,"[eugenie, grandet, i, there, are, houses, in, ...","Balzac, Honoré de"
4,"[within, these, houses, there, is,, perhaps,, ...","Balzac, Honoré de"


## Train doc2vec word model

In [61]:
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(df['sentence'].values)]

max_epochs = 5
vec_size = 20
alpha = 0.025

model = Doc2Vec(size=vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                dm =1)

model.build_vocab(documents)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(documents,
                total_examples=model.corpus_count,
                epochs=model.iter)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

iteration 0


C:\Users\Administrator\anaconda3\lib\site-packages\ipykernel_launcher.py:19: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).


iteration 1
iteration 2


In [62]:
model.infer_vector(df.iloc[20]['sentence'])

array([-0.02505612,  0.11131589,  0.04942008,  0.02194801,  0.07401586,
       -0.01383335,  0.00202945,  0.06942933,  0.02606135, -0.06500115,
       -0.02738094,  0.08196763, -0.02487132, -0.07051333, -0.00149486,
       -0.06700415,  0.12313438, -0.08620954, -0.08924749,  0.00532404],
      dtype=float32)

In [64]:
f = open("wv.pickle","wb")
pickle.dump(model, f)
f.close()

## Label authors

In [65]:
# [0,15,13,7,3,...]
author_list = list(set(df['author']))
label = []
for i in df['author']:
    label.append(author_list.index(i))

In [76]:
word_model.docvecs.most_similar(500)

[(1258106, 0.9939539432525635),
 (1513655, 0.993355393409729),
 (1265277, 0.9914419054985046),
 (1366326, 0.9907000660896301),
 (1293825, 0.989872395992279),
 (1709502, 0.9895474910736084),
 (1415846, 0.989281415939331),
 (421158, 0.9888761639595032),
 (1699449, 0.9888172149658203),
 (1834791, 0.9886716604232788)]

In [77]:
X = []
for i in df['sentence']:
    X.append(model.infer_vector(i))

In [78]:
len(X)

1904848

## Pickle the 1.8M word feature

In [79]:
f = open("wv_X.pickle","wb")
pickle.dump(X, f)
f.close()